In [ ]:
!pip install geopandas dask

In [ ]:
from pathlib import Path
from zipfile import ZipFile

import geopandas as gpd
import httpx
import rioxarray
from rich.progress import track

from jupytergis.tiler import GISDocument

Please run the `south_america.ipynb` notebook in order to download the DEM data.

In [ ]:
url = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_dem_30s_grid.zip"
filename = Path(url).name
name = filename[: filename.find("_grid")]
adffile_dem = Path(name) / name / "w001001.adf"
da_dem = rioxarray.open_rasterio(adffile_dem, masked=True)
da_dem = da_dem.sel(band=1)
da_dem = da_dem.chunk(dict(x=1000, y=1000))
vmin_dem, vmax_dem = int(da_dem.min().compute()), int(da_dem.max().compute())
vmin_dem, vmax_dem

Let's download some watershed shapefiles for South America.

In [ ]:
url = "https://data.hydrosheds.org/file/hydrobasins/standard/hybas_sa_lev01-12_v1c.zip"
filename = Path("hybas_sa_lev01-12_v1c.zip")

if not filename.exists():
    with httpx.stream("GET", url) as r, open(filename, "wb") as f:
        total = int(r.headers["Content-Length"]) / 1024
        for data in track(r.iter_bytes(chunk_size=1024), total=total, description="Downloading"):
            f.write(data)
        f.flush()
    zip = ZipFile(filename)
    zip.extractall("hybas_sa_lev01-12_v1c")

The data consists of multiple levels of watersheds.

In [ ]:
shape = gpd.read_file("hybas_sa_lev01-12_v1c/hybas_sa_lev03_v1c.shp")
shape.plot()

We will use the Amazon river watershed to clip the DEM data.

In [ ]:
geometry = shape.iloc[6].geometry
geometry

In [ ]:
geodf = gpd.GeoDataFrame(geometry=[geometry], crs="EPSG:4326")

In [ ]:
Path("clipping.jGIS").unlink(missing_ok=True)
doc = GISDocument("clipping.jGIS")
doc

In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr
from rio_tiler.models import ImageData
from titiler.core.algorithm import BaseAlgorithm


class ClipAlgo(BaseAlgorithm):
    def __call__(self, img: ImageData) -> ImageData:
        data = img.data[0]
        dx = (img.bounds.right - img.bounds.left) / data.shape[1]
        dy = (img.bounds.top - img.bounds.bottom) / data.shape[0]
        x = [img.bounds.left + i * dx for i in range(data.shape[1])]
        y = [img.bounds.top - i * dy for i in range(data.shape[0])]
        coords = {"x": x, "y": y}
        data_array = xr.DataArray(data, dims=("y", "x"), coords=coords)
        data_array.rio.write_crs("epsg:3857", inplace=True)
        try:
            clipped = data_array.rio.clip(geodf.geometry.values, geodf.crs, drop=False).values
        except BaseException:
            clipped = np.zeros_like(data)
        return ImageData(
            clipped,
            assets=img.assets,
            crs=img.crs,
            bounds=img.bounds,
        )

In [ ]:
await doc.add_tiler_layer(
    name="Digital elevation model layer",
    data_array=da_dem,
    colormap_name="terrain",
    rescale=(vmin_dem, vmax_dem),
    algorithm=ClipAlgo,
)

In [ ]:
# not mandatory, but helps shutting down the kernel gracefully:
# await doc.stop_tile_server()